In [1]:
import rasterio   # 导入rasterio库，用于读取和处理栅格数据
from rasterio.warp import calculate_default_transform, reproject, Resampling  # 导入重投影相关的函数和枚举
import geopandas as gpd  # 导入geopandas库，用于处理矢量数据（GeoDataFrame）
import numpy as np
import os  # 导入操作系统模块，用于与操作系统进行交互（如文件路径处理）
os.environ['PROJ_LIB'] = r'C:/Users/HP/miniconda3/envs/venv/Lib/site-packages/pyproj/data'  # 设置PROJ_LIB环境变量

In [2]:
# 定义重投影函数
def img_reproj(data_rio, ref_data=None, crs_dst=None, output_path=None):
    """
    重投影栅格数据函数。
 
    参数:
    data_rio: rasterio类对象，表示待重投影的栅格数据。
    ref_data: 参考数据，可以是栅格数据、rasterio对象或矢量数据（GeoDataFrame），提供重投影坐标系。
    crs_dst: epsg格式的坐标系，用于指定重投影的目标坐标系（如果ref_data提供，则此参数可选）。
    output_path: 保存重投影后数据的路径。
 
    返回:
    data_reproj_rio: 重投影后的rasterio栅格对象。
    """
 
    # 检查ref_data和crs_dst参数，确保至少有一个被设置
    if ref_data is None and crs_dst is None:
        raise ValueError("必须设置ref_data或crs_dst参数之一")
 
    # 提取参考数据的坐标系
    if isinstance(ref_data, str):  # ref_data是文件路径
        if ref_data.lower().endswith(('.tif', '.tiff')):  # 参考数据为栅格数据
            with rasterio.open(ref_data) as ref_rio:
                crs_dst = ref_rio.crs
        else:  # 参考数据为矢量数据
            gdf = gpd.read_file(ref_data)
            crs_dst = gdf.crs.to_string()
    elif isinstance(ref_data, rasterio.io.DatasetReader):  # rasterio对象
        crs_dst = ref_data.crs
    elif isinstance(ref_data, gpd.GeoDataFrame):  # GeoDataFrame对象
        crs_dst = ref_data.crs.to_string()
 
    # 获取原始栅格数据的坐标系
    crs_src = data_rio.crs
 
    # 计算重投影所需的变换参数
    transform, width, height = calculate_default_transform(
        crs_src, crs_dst, data_rio.width, data_rio.height, *data_rio.bounds
    )
 
    # 创建一个新的rasterio对象，用于存储重投影后的数据
    kwargs = data_rio.meta.copy()
    kwargs.update({
        'crs': crs_dst,
        'transform': transform,
        'width': width,
        'height': height
    })
 
    # 重投影数据
    with rasterio.Env():
        with rasterio.open(data_rio.name) as src:
            if output_path:
                # 如果指定了输出路径，则直接保存到磁盘
                with rasterio.open(output_path, 'w', **kwargs) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=crs_dst,
                            resampling=rasterio.warp.Resampling.bilinear
                        )
                    return None  # 不返回rasterio对象，因为已经保存到磁盘
            else:
                # 如果没有指定输出路径，则保存到内存并返回
                data_reproj_rio = rasterio.MemoryFile()
                with data_reproj_rio.open(**kwargs) as memfile:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(memfile, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=crs_dst,
                            resampling=rasterio.warp.Resampling.bilinear
                        )
                return data_reproj_rio



In [3]:
path_1 = 'data/Section-6/s2_kunming_chenggong_6bands_20m.tif'
path_out = 'data/Section-6/s2_kunming_chenggong_6bands_20m_wgs84.tif'
vec_ref = 'data/Section-6/kunming_wgs84.gpkg'
raster_ref = 'data/Section-6/s2_kunming_chenggong_4bands_10m_wgs84.tif'

data_rio = rasterio.open(path_1)  # 打开栅格数据文件
vec_ref_gpd = gpd.read_file(vec_ref)  # 加载 GPKG 矢量数据
data_ref_rio = rasterio.open(raster_ref)  # 打开栅格数据文件

crs_dst = 'EPSG:4326'  # 设置目标CRS为WGS84坐标系
##示例调用
dst = img_reproj(data_rio=data_rio, crs_dst=crs_dst, output_path=path_out)   # 调用重投影函数


CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: C:\Users\HP\miniconda3\envs\venv\Library\share\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.